#Gold Layer Scripting : Business-Level Data Modeling

This script creates views for the Gold layer in the data lakehouse.
The Gold layer represents the final dimension and fact tables.Each view performs transformations and combines data from the Silver layer
to produce a clean, enriched, and business-ready dataset, that answer real business questions.

### The Transformation Logic : 

Create a 360° customer table for reporting and segmentation.The Customer Dimension consolidates customer data from multiple systems into a single, unified table. It combines CRM customer details with ERP customer and location information to provide enriched attributes for analytics.

In [0]:
querry = """

select  
  row_number() over (order by ci.customer_id) as customer_key ,
  ci.customer_id ,
  ci.customer_number ,
  ci.first_name ,
  ci.last_name ,
  ci.marital_status ,
  la.country ,
  case 
    when ci.gender <> 'n/a' then ci.gender
    else coalesce(cu.gender ,'n/a')
  end as gender,
  cu.birth_date as birthdate,
  ci.created_date

from 
  bike_lakehouse.silver.crm_customers as ci

left join 
  bike_lakehouse.silver.erp_customers as cu
on
  ci.customer_number = cu.customer_number

left join
  bike_lakehouse.silver.erp_customer_location as la 
on
  ci.customer_number = la.customer_number

"""


In [0]:
df = spark.sql(querry)

In [0]:
df.limit(10).display()

customer_key,customer_id,customer_number,first_name,last_name,marital_status,country,gender,birthdate,created_date
1,11000,AW00011000,Jon,Yang,Married,Australia,Male,1971-10-06,2025-10-06
2,11001,AW00011001,Eugene,Huang,Single,Australia,Male,1976-05-10,2025-10-06
3,11002,AW00011002,Ruben,Torres,Married,Australia,Male,1971-02-09,2025-10-06
4,11003,AW00011003,Christy,Zhu,Single,Australia,Female,1973-08-14,2025-10-06
5,11004,AW00011004,Elizabeth,Johnson,Single,Australia,Female,1979-08-05,2025-10-06
6,11005,AW00011005,Julio,Ruiz,Single,Australia,Male,1976-08-01,2025-10-06
7,11006,AW00011006,Janet,Alvarez,Single,Australia,Female,1976-12-02,2025-10-06
8,11007,AW00011007,Marco,Mehta,Married,Australia,Male,1969-11-06,2025-10-06
9,11008,AW00011008,Rob,Verhoff,Single,Australia,Female,1975-07-04,2025-10-06
10,11009,AW00011009,Shannon,Carlson,Single,Australia,Male,1969-09-29,2025-10-06


###Writing Gold Table
Persist the cleaned DataFrame as a Delta table in the Gold layer.

In [0]:
df.write.mode('overwrite').format('delta').saveAsTable('bike_lakehouse.gold.dim_customers')

###Sanity checks of Gold table
Quickly check the result of transformations, before moving forward with the dataFrame

In [0]:
%sql
select * from bike_lakehouse.gold.dim_customers limit 10 ;

customer_key,customer_id,customer_number,first_name,last_name,marital_status,country,gender,birthdate,created_date
1,11000,AW00011000,Jon,Yang,Married,Australia,Male,1971-10-06,2025-10-06
2,11001,AW00011001,Eugene,Huang,Single,Australia,Male,1976-05-10,2025-10-06
3,11002,AW00011002,Ruben,Torres,Married,Australia,Male,1971-02-09,2025-10-06
4,11003,AW00011003,Christy,Zhu,Single,Australia,Female,1973-08-14,2025-10-06
5,11004,AW00011004,Elizabeth,Johnson,Single,Australia,Female,1979-08-05,2025-10-06
6,11005,AW00011005,Julio,Ruiz,Single,Australia,Male,1976-08-01,2025-10-06
7,11006,AW00011006,Janet,Alvarez,Single,Australia,Female,1976-12-02,2025-10-06
8,11007,AW00011007,Marco,Mehta,Married,Australia,Male,1969-11-06,2025-10-06
9,11008,AW00011008,Rob,Verhoff,Single,Australia,Female,1975-07-04,2025-10-06
10,11009,AW00011009,Shannon,Carlson,Single,Australia,Male,1969-09-29,2025-10-06
